# 월간식당 빅데이터 분석

* 키워드별 단어 출현 빈도

In [1]:
#!pip install konlpy xlrd openpyxl

In [2]:
import re
import numpy as np
import pandas as pd
import json
import string
from pandas import read_excel

import warnings
warnings.simplefilter("ignore")

from konlpy.utils import pprint
from konlpy.tag import Hannanum
hannanum = Hannanum()

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

from msba import posts as p
from msba import stopwords as stopswords

## 키워드/관심어 불러오기

In [3]:
def interested_words():
    # 핵심단어 읽어 오기
    my_sheet = '소비키워드'
    keywords_filename = 'deskresearch_.xlsx'
    df = read_excel(keywords_filename, sheet_name = my_sheet, header=1) # index_col='번호'
    keywords = df['핵심단어']
    subkeywords = df['대체어']
    interested_words = df['키워드']
    return keywords, subkeywords, interested_words

keywords, subkeywords, interested_words = interested_words()

for keyword, subkeyword, interested_word in zip(keywords, subkeywords, interested_words):
    subkeyword = subkeyword.replace(" ", "").replace(",","|")
    interested_word = subkeyword + "|" + interested_word.replace(" ", "").replace(",","|")
#     print(keyword, " : " , subkeyword, interested_word)

# 불용어

In [4]:
# word = [ '1인미디어' ]
stopwords = stopswords.stopwords
stopwords_kr = stopwords + [
'하지만', '그리고', '그런데', '저는','제가','있습니다', '않습니다', 'ㅋㅋ', 'ㅋㅋㅋ', 'ㅎㅎ', 'ㅎㅎㅎ',
'그럼', '이런', '저런', '합니다', '있어요', '참고로', '그러고', '아시죠', '하는',
'있어서', '그냥', '같아요', '입니다', 'com', '아주', '않습니다',
'같은', '해서', '있고', '않고', '없는', '있는데',
'www', '먼저', '다시', '있도록', '등을','대한','있으니','미리','것이','모든', '00','없이',
'정도','오늘', '근데', '같습니다','통해', '내가','나는', '나오는', 'http', '바랍니다',
'있어', '위한','요즘','한번','밝혔다','계속', '됩니다','사실', '더욱', '하기',
'아닌', '쉬게', '여러', '가능합니다', '보고', '분들이', '있다는', '라는', '등의',
'수도', '되었습니다', '부탁드립니다', '있는데요', '되고', '있게', '이를', '통한', '것으로', '있으며', '가지', 
'경우', '000원','이미','가서','지금','by','그런', '보니','on', '했는데', '전에','좋을',
'위해', '만원', '인분', '인당', '배가', '완전', '자주', '양이', '오늘은', '먹었어요', '처음', '되는', '좋아요',
'먹은', '것은', '먹으니', '높은', '정도로', '넣고', 'ㅠㅠ', '나왔어요', 'ㅋㅋㅋㅋ', '아직',
'가는', '것도', '것을','있다고', '기자', '따른','보면','알고','있을','11','10','19',
'주는', '되면', '매우', '보다', '대신', '때문이다', '가진', '이는', '출처', '우리는', '앞으로', 
'했다', '곳이', '한다', '이렇게', '하고', '된다', '매우', '의한', '설명하시오', '아래와', '이후', '교시', '들어', '있지만',
'것이다', '출처', '나의', '말합니다', '나를', '된다', '인해', '가능한', '관련', '하지', '없다', '비건', 
'image', '이제', '라고','있었어요','해도','좋습니다', '없고','위에','나온', '않는', '가지고','않은','그렇게','하는데',
'그런지','많아서','것입니다','이건','또는', '있었습니다', '되어', '없습니다', '싶은',
'이거', '사용할', '전혀', '아예', '표시되지', '살짝', '오는', '다들', '밖에', '요즘은', 
'개의', 'ㅋㅋㅋㅋㅋ', '안에', '이상의', '생각합니다', '넘어갑니다더싸다특가', '등으로', '가면', '파는',
'하네요', '넣어', '하니', '하네요', '사용한', '개의', '년까지', '했습니다', '하게', '하니', '하네요', '있기', '있으면', 
'아니다', '같아서', '먹일', '것이라는', '보인다', '사상', '만명', 
'않는다', '있다면', '하는데요', '하니', '하네요', '일부터', '뭔가', '와의', '것이라', '아닐까', '주고',
'것에', '것이라고', '그것을', '누구나', '두고', '싶습니다', '중에', '못하는', '것이죠', '한다는', '그렇다면', 
'봅니다', '수밖에', '싶네요', '맛있어요', '맛있었어요', '올려서', '오랜만에', '주문했어요', '우리가', '사람이',
'그의', '헤헤헤', '기반', '그는', '이라는', '이다', '말했다', '그들의', '주로', '이번에', '아래', '만큼', '억원',
'받고', '해야', '있죠', '받을', '오전', '오후', '나타났다', '반면', '뜻하는', '대상으로', '관계자는', 
'갖고', '다음과', '달러', '먹으면', '아니고', '있음', '예상', '경우가', '원하는', '뜻하는', '이제는', '년간', 
'와서', '이게', '있었다', '지난', '사용하는', '번째', '등이', '이하', '인한', '이에', '서비스입니다', '있는', '그대로', 
'있답니다', '일까지', '위치한', '대해', '번째', '먹고', '먹기', '먹는', '먹을', '마시고', '먹어도', '먹었는데', '먹어야', '먹었던',
'마시는', '마시면', '가장', '엄청', '살짝', '같은데', '넘나', 
'확실히', '등은', '지난달', '대부분', 
'하면', 'photo','아니라','역시','저희는', '저도','특히','30', '따로', '안녕하세요','항상','현재',
'많은', '많이', '정말', '너무', 'https', 'Coupang', 'Coupa', 'ng'
] 



# 자신의 관심사에 맞는 단어로 데이터 가져오기

In [5]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    text = re.sub('[A-z]', '', text)
    text = re.sub('[0-9]', '', text)
    res = ''.join([i for i in text if not i.isdigit()]) 
    return text

def preprocessing_2(rows):
    pattern = re.compile(r"[.,?!★~]")
    sentences = []
    for row in rows:
        sentences = sentences + pattern.split(row.replace("\xa0", "").replace("\t","").strip())
        sentences = list(set(sentences))        
    return sentences

def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

# 실전 : 말뭉치 생성 및 전처리
def preprocessing_3(sentences):
    # Convert list of strings to string
#     full_str = convert_list_to_string(sentences[1])
#     # print(full_str)

    pattern = re.compile(r".*(광고정보).*")

    corpus = []
    for sentence in sentences:
        if len(sentence):
            sentence = sentence.strip().split(" ")
            sentence = convert_list_to_string(sentence).replace("  ", " ").strip()
            if pattern.match(sentence):
                continue
            else:
                corpus.append(sentence)
    return corpus

def preprocessing_4(docs):
    # docs = [
    #         w for w in hannanum.nouns(" ".join(cell)) for cell in cells
    #         if ((not w[0].isnumeric()) and (w[0] not in string.punctuation))
    # ]
    vect = CountVectorizer(stop_words=stopwords_kr, min_df=5, max_df=200).fit(docs)
    count = vect.transform(docs).toarray().sum(axis=0)
    idx = np.argsort(-count)
    count = count[idx]
    feature_name = np.array(vect.get_feature_names())[idx]
    # plt.bar(range(len(count)), count)
    # plt.show()

    tf_list = list(zip(feature_name, count))[:100]

    tf_df = pd.DataFrame(tf_list,columns=['단어', '빈도'])
    return tf_df

def save_to_csv(keyword, tf_df):
    # save to csv
    filename = "./output/" + "최빈어_" + keyword.replace(" ","") + ".csv"   
    # filename_list.append(filename)
    tf_df.to_csv(filename, date_format='%Y%m%d', encoding='utf-8-sig')

In [6]:
def interested_words():
    # 핵심단어 읽어 오기
    my_sheet = '소비키워드'
    keywords_filename = 'deskresearch_.xlsx'
    df = read_excel(keywords_filename, sheet_name = my_sheet, header=1) # index_col='번호'
    keywords = df['핵심단어']
    subkeywords = df['대체어']
    interested_words = df['키워드']
    return keywords, subkeywords, interested_words


In [7]:
searchkeys = []
keywords, subkeywords, interested_words = interested_words()

for keyword, subkeyword, interested_word in zip(keywords, subkeywords, interested_words):
    subkeyword = subkeyword.replace(" ", "").replace(",","|")
    interested_word = subkeyword + "|" + interested_word.replace(" ", "").replace(",","|")
    searchkey = keyword + "|" + subkeyword.replace(",","|") + "|" + interested_word
    searchkeys.append(searchkey)

In [8]:
searchkeys

['1인 외식|혼밥|혼밥|1인피자|소포장|싱글족|혼술|나홀로시장|HOLO|Healthcare|Oversize|lifeathome|onlineshopping|간편프리미엄식|1인좌석|1인메뉴|누들바',
 '가치소비|펀슈머|펀슈머|미코노미|가성비|박리다매|노브랜드|가심비|가잼비|펀슈머|잔술|나심비|가시비|가성비갈빗집|가성비고기집|중저가곱창|저가형양대창',
 'RMR|밀키트|밀키트|밀키트|쿡킷|HMR|맛집간편식|냉동식품|간편식',
 '웰빙|글루텐프리|글루텐프리|건강식|로가닉|글루텐프리|설탕대체제|다이어트|유산균|두부|웰빙음식|슈퍼푸드|마인드풀이팅|이너뷰티|웰빙=힙',
 '체험소비|라이프스타일숍|라이프스타일숍|라이프스타일공간|북테리어|라이브러리카페|복합문화공간|베이커리카페|북카페|체험마케팅|고객경험',
 '골목상권|힙지로|힙지로|힙지로|젠트리피케이션|골목맛집|송리단길|망리단길',
 '공유경제|공유주방|공유주방|공간대여|공간재사용|점포셰어링|고스트키친|가상식당|가상주방|공유주방|클라우드키친',
 '구독서비스|큐레이션|큐레이션|구독경제|정기구독서비스|정기고객|큐레이션서비스',
 '로컬푸드|로컬맛집|로컬맛집|로컬맛집|로컬와인|리얼로컬|빵순례|지방브랜드|신토불이|지역특산물|라스트마일|푸드마일리지',
 '모디슈머|괴랄미|괴식|괴랄미|괴식|트러플짜파게티|한우채끝살짜파구리|괴랄미|괴식|꿀주|뚱카롱|덕력|아샷추|달고나커피|귀띠머신|야민정음',
 '1인 미디어|리뷰투버|먹방|리뷰투버|먹방|SNS|먹방|유투브|ASMR|인플루언서|인스타그래머블|랜선라이프|라이브컨텐츠|라이브커머스|리뷰튜버|미디어커머스',
 '배달음식|배달앱|배달앱|배달플랫폼|배달대행|배달서비스|드라이브스루|공공배달앱|배달앱|야식배달|배달빙수|배달커피|배달전문|온라인맛집|온라인플랫폼|새벽배송|온라인구매',
 '푸드홀|복합매장|복합매장|몰링컬쳐|복합쇼핑몰|복합문화공간|몰링족',
 '스내킹 스토어|포터블푸드|포터블푸드|포터블푸드|이동성|그로서란트|그랩앤고|스내킹카페|도시락편집매장',
 '스몰 럭셔리|작은사치|샴페인바|작은사치|

In [9]:
lst = []
for searchkey in searchkeys:
    lst.append(searchkey.replace(" ","").split("|"))

In [10]:
import itertools
def oneDArray(x):
    return list(itertools.chain(*x))

In [11]:
lst = oneDArray(lst)

In [12]:
for w in lst:
    print (w + "\t" + "ncn")

1인외식	ncn
혼밥	ncn
혼밥	ncn
1인피자	ncn
소포장	ncn
싱글족	ncn
혼술	ncn
나홀로시장	ncn
HOLO	ncn
Healthcare	ncn
Oversize	ncn
lifeathome	ncn
onlineshopping	ncn
간편프리미엄식	ncn
1인좌석	ncn
1인메뉴	ncn
누들바	ncn
가치소비	ncn
펀슈머	ncn
펀슈머	ncn
미코노미	ncn
가성비	ncn
박리다매	ncn
노브랜드	ncn
가심비	ncn
가잼비	ncn
펀슈머	ncn
잔술	ncn
나심비	ncn
가시비	ncn
가성비갈빗집	ncn
가성비고기집	ncn
중저가곱창	ncn
저가형양대창	ncn
RMR	ncn
밀키트	ncn
밀키트	ncn
밀키트	ncn
쿡킷	ncn
HMR	ncn
맛집간편식	ncn
냉동식품	ncn
간편식	ncn
웰빙	ncn
글루텐프리	ncn
글루텐프리	ncn
건강식	ncn
로가닉	ncn
글루텐프리	ncn
설탕대체제	ncn
다이어트	ncn
유산균	ncn
두부	ncn
웰빙음식	ncn
슈퍼푸드	ncn
마인드풀이팅	ncn
이너뷰티	ncn
웰빙=힙	ncn
체험소비	ncn
라이프스타일숍	ncn
라이프스타일숍	ncn
라이프스타일공간	ncn
북테리어	ncn
라이브러리카페	ncn
복합문화공간	ncn
베이커리카페	ncn
북카페	ncn
체험마케팅	ncn
고객경험	ncn
골목상권	ncn
힙지로	ncn
힙지로	ncn
힙지로	ncn
젠트리피케이션	ncn
골목맛집	ncn
송리단길	ncn
망리단길	ncn
공유경제	ncn
공유주방	ncn
공유주방	ncn
공간대여	ncn
공간재사용	ncn
점포셰어링	ncn
고스트키친	ncn
가상식당	ncn
가상주방	ncn
공유주방	ncn
클라우드키친	ncn
구독서비스	ncn
큐레이션	ncn
큐레이션	ncn
구독경제	ncn
정기구독서비스	ncn
정기고객	ncn
큐레이션서비스	ncn
로컬푸드	ncn
로컬맛집	ncn
로컬맛집	ncn
로컬맛집	ncn
로컬와인	ncn
리얼로컬	ncn
빵순례	ncn
지방브랜드	ncn
신토불이	ncn
지역특산물	ncn
라스트마일	ncn
푸드마일리

In [ ]:
for keyword, subkeyword, interested_word in zip(keywords, subkeywords, interested_words):
    
    df = p.readall(keyword.replace(" ",""))
    df = df[ (df['date'] >= '2019-07-01') & (df['date'] < '2020-07-01')]
    df = df.drop_duplicates()
    # print(df.shape)  
    rows = df['title'].apply(preprocessing) + df['content'].apply(preprocessing)
    # print(len(rows), type(rows))
    sentences = preprocessing_2(rows)
    # print(len(sentences), type(sentences))
    corpus = preprocessing_3(sentences)
    # print(len(corpus), type(corpus))
    
    tf_df = preprocessing_4(corpus)
    save_to_csv(keyword, tf_df)